## KAKENからダウンロードしたXMLファイルをパースして、ローカルのMariaDBに保存するプログラム

### ファイル構成

4つのファイルでできています。部品を作る3つのファイルと、3つの部品を統合するファイルです。

#### 部品を作る
- parse_grantaward_main.ipynb
 - 研究課題のメインになる部分。課題番号、研究種目、開始年度、終了年度、直接経費金額など
- parse_grantaward_institution_from_grantlist.ipynb　←いまここ
 - 採択年度の研究機関
- parse_grantaward_member_from_summary.ipynb
 - 採択年度の研究代表者

#### 課題番号をキーにして、3つの部品を統合して一つのテーブルを作る
- parse_grantaward_integration.ipynb

### 事前準備
- kaken_zenkadai_download.ipynbを実行して、./xmlフォルダにXMLファイルを保存しておいてください。
- ここでは、2010年以降のデータを扱うことにしています。

### 使い方

- 部品を作る3つのファイルを適宜の順序で実行します。それぞれから、./afterCleaningフォルダにデータフレーム（部品）が作られ、pickle形式でファイルを保存します。
 - parse_kadai.dump
 - parse_institution_from_grantlist.dump
 - parse_member_from_summary.dump
- 3つの部品ができたら、parse_grantaward_integration.ipynbを実行すると、一つのテーブルに結合してローカルのMariaDBに保存されます。

### 今後の予定

自分が眺めた範囲では、古い年代ほどデータの欠損など問題があって、前処理が必要な雰囲気なので、新しいところから始めました。時間をみつけて、2010年以前のデータもパースできるようにしたいと思っています。KAKENに研究者番号が入っているのが1985年以降なので、優先順位としてはそこが一つの境目になると思っています。84年以前は、必要に応じてやりましょう。

In [ ]:
# encoding: utf-8
from lxml import etree
from tqdm import tqdm_notebook as tqdm
import pandas as pd
import numpy as np
import pickle
import glob
import re

## 研究機関を抽出する関数

採択時点の研究機関を知りたい。研究機関の情報は、summaryとgrantlistにある。summaryの研究機関は課題終了時の情報になっており、grantlistには年度ごとの研究機関の情報がある。ここではgrantlistから課題ごとに最も古い年度の研究機関データを取得する

In [ ]:
def institution(xmlfile):
    tree = etree.parse(xmlfile)
    nsmap = {"xml": "http://www.w3.org/XML/1998/namespace"}
    institutionlist = []
    for grantAward in tree.iterfind("grantAward"):
        awardnumber = grantAward.get("awardNumber")
        if grantAward.find("grantList/grant[@xml:lang='ja']", nsmap) is None:
            row = [
                awardnumber,
                np.NaN,
                np.NaN,
                np.NaN,
                np.NaN,
                np.NaN,
                np.NaN,
                np.NaN,
            ]
            institutionlist.append(row)
        else:
            for grant in grantAward.find("grantList").iterfind("grant[@xml:lang='ja']", nsmap):
                fiscalyear = grant.get("fiscalYear")
                grant_sequence = grant.get("sequence")
                if grant.find("institution") is None:
                    row = [
                        awardnumber,
                        fiscalyear,
                        grant_sequence,
                        np.NaN,
                        np.NaN,
                        np.NaN,
                        np.NaN,
                        np.NaN,
                    ]
                    institutionlist.append(row)

                else:
                    for institution in grant.iterfind("institution"):
                        institution_sequence = institution.get("sequence")
                        institution_niicode = institution.get("niiCode")
                        institution_mextcode = institution.get("mextCode")
                        institution_jspscode = institution.get("jspsCode")
                        institution = institution.text

                        row = [
                            awardnumber,
                            fiscalyear,
                            grant_sequence,
                            institution_sequence,
                            institution_niicode,
                            institution_mextcode,
                            institution_jspscode,
                            institution,
                        ]
                        institutionlist.append(row)
        

    df = pd.DataFrame(institutionlist)
    df.columns = [
        'awardnumber',
        'fiscalyear',
        'grant_sequence',
        'institution_sequence',
        'institution_niicode',
        'institution_mextcode',
        'institution_jspscode',
        'institution',
    ]
    
    pickledfile = 'pickledDF_institution_from_grantlist/' + re.search('[0-9]{4}_[0-9]+-[0-9]+.xml', xmlfile).group() + '.dump'
    df.to_pickle(pickledfile)

フォルダをいったんきれいにする関数を定義して、実行する

In [ ]:
import os
import shutil

def cleandir(dirname):
    if os.path.isdir(dirname):
        shutil.rmtree(dirname)
    os.mkdir(dirname)

cleandir('pickledDF_institution_from_grantlist')

関数を実行して、XMLファイルごとにパースして、pickleして保存する

In [ ]:
for xmlfile in tqdm(glob.glob('xml/201*.xml')):
    institution(xmlfile)

保存したpickleを読み込んで、データフレームを全部連結する

In [ ]:
columns = [
    'awardnumber',
    'fiscalyear',
    'grant_sequence',
    'institution_sequence',
    'institution_niicode',
    'institution_mextcode',
    'institution_jspscode',
    'institution',
]
df = pd.DataFrame(columns=columns)

for dump in tqdm(glob.glob('pickledDF_institution_from_grantlist//*.dump')):
    with open(dump, mode='rb') as f:
        df = pd.concat([df, pickle.load(f)])

連結したデータフレームをいったんpickleして保存

In [ ]:
df.to_pickle('beforeCleaning/parse_institution_from_grantlist.dump')

### データクリーニング

pickleからデータフレームを復元する

In [ ]:
with open('beforeCleaning/parse_institution_from_grantlist.dump', mode='rb') as f:
    df = pickle.load(f)

In [ ]:
df.info()

nullを0で置換する

In [ ]:
df = df.fillna(0)
df.info()

intに指定

In [ ]:
df = df.astype({
    'fiscalyear': np.int64,
    'grant_sequence': np.int64,
    'institution_sequence': np.int64,
    'institution_niicode': np.int64,
    'institution_mextcode': np.int64,
    'institution_jspscode': np.int64,    
})
df.info()

awardnumberごとにfiscalyearが最小の行を取得する。

In [ ]:
oldest = df.groupby('awardnumber')['fiscalyear'].min().reset_index()
oldest

institutionを連結する

In [ ]:
df = pd.merge(oldest, df, on=['awardnumber', 'fiscalyear'])
df

重複したデータがあるかどうか

In [ ]:
df.duplicated().any()

重複はなかった（あったら何かが違う）。

awardnumberがユニークかどうか確認するために、行数255813とawardnumberのユニークな値の数が一致するかどうか確認する。

In [ ]:
df['awardnumber'].nunique(dropna=False)

awardnumberはユニークだったので（ユニークじゃなかったら何かが違う）、インデックスにする。

In [ ]:
df = df.set_index('awardnumber')
df

grantsequenceなど、各列のデータの様子を概観する

In [ ]:
pd.crosstab(df['grant_sequence'], df['fiscalyear'], margins=True)

In [ ]:
pd.crosstab(df['institution_sequence'], df['fiscalyear'], margins=True)

In [ ]:
pd.crosstab(df['institution_niicode'], df['fiscalyear'], margins=True)

In [ ]:
pd.crosstab(df['institution_mextcode'], df['fiscalyear'], margins=True)

In [ ]:
pd.crosstab(df['institution_jspscode'], df['fiscalyear'], margins=True)

niicode、mextcode、jspscodeで比較して、コード0が最も少ないのはniicodeだった。これからniicodeを使うことにする。

研究機関名institutionとniicodeの対応状況を見る。たとえば京大だと…

In [ ]:
kyodai = df[df['institution_niicode'] == 14301]
pd.crosstab(kyodai['institution'], kyodai['institution_niicode'])

研究機関名institutionとniicodeの対応状況を見る。たとえば東大だと…

In [ ]:
todai = df[df['institution_niicode'] == 12601]
pd.crosstab(todai['institution'], todai['institution_niicode'])

In [ ]:
df.query('institution_niicode == 12601 & institution == "秋田県立大学"')

In [ ]:
df.query('institution_niicode == 12601 & institution == "鹿児島大学"')

数件違うけど、調べてみたら、内定の前後で異動している人のようだ。niicodeをベースにすることにする。

ここまでで中間データをいったんpickleして保存

In [ ]:
df.to_pickle('afterCleaning/parse_institution_from_grantlist.dump')

ひとまず終了。

次は、parse_grantaward_member_from_summary.ipynbで内定時の研究代表者を取得する。